In [1]:
import cv2
import mediapipe as mp
import pickle
import math
import os
import numpy as np
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# media pipe pose 전역변수로 설정
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=True,min_detection_confidence=0.3, model_complexity=0)
mp_drawing = mp.solutions.drawing_utils

In [3]:
# 포즈 감지 함수
def detectPose(image,pose,display=True):
    # input 이미지 복사
    output_image = image.copy()
    # RGB로 변환
    imageRGB = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    # 렌드마크 찍기
    results = pose.process(imageRGB)
    # 사진 사이즈 구하기
    height, width, _ = image.shape
    # 렌드마크 값 리스트 만들기
    landmarks = []

    if results.pose_landmarks:
        # 렌드마크 그리기
        mp_drawing.draw_landmarks(image=output_image, landmark_list=results.pose_landmarks, connections=mp_pose.POSE_CONNECTIONS)
        #렌드마크 정규값 landmarks 리스트에 넣기
        for landmark in results.pose_landmarks.landmark:

            landmarks.append((int(landmark.x*width), int(landmark.y*height), (landmark.z*width)))
    # 사진 결과 보기
    if display:
        plt.figure(figsize=[22,22])
        plt.subplot(121);plt.imshow(image[:,:,::-1]);plt.title("Original Image");plt.axis('off');
        plt.subplot(122);plt.imshow(output_image[:,:,::-1]);plt.title("Output Image");plt.axis('off');
        # 3차원으로도 봐보기
        mp_drawing.plot_landmarks(results.pose_world_landmarks, mp_pose.POSE_CONNECTIONS)
    else:
    # diaplay 에 false 입력시 이미지랑 렌드마크 정보 반환
        return output_image, landmarks

In [4]:
# 각도 구하는 함수
def calculateAngle(landmark1, landmark2, landmark3):
    x1,y1,_ = landmark1 # z 좌표는 사용 x
    x2,y2,_ = landmark2
    x3,y3,_ = landmark3
    # 3점의 각도 -> 선 2개 -> 각도
    angle = math.degrees(math.atan2(y3-y2, x3-x2) - math.atan2(y1-y2,x1-x2))

    angle = np.abs(angle)
    # 음수의 각도가 나오면 양수로 바꾸기
    if angle >180.0:
        angle = 360 - angle
        
    return angle

In [5]:
# 테스트 할 영상 경로 넣으면 됨
video = cv2.VideoCapture()

In [ ]:
counter = 0
stage = 'start'
a = [] #시작각도
b = [] #고점
c = [] #1회 끝 (다시 시작 각도)
df = pd.DataFrame() # predict에 넣어 줄 X_test
while video.isOpened():
    # Read a frame.
    ok, frame = video.read()
    # Check if frame is not read properly.
    if not ok:
        # Break the loop.
        break
    # display -> False 넣어서 img랑 렌드마크 받기
    output_img, landmarks = detectPose(frame,pose,display=False)

    if landmarks:
         # 귀 - 어깨 - 엉덩이 => 고개 각도 구하기 위함
        left_shoulder_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.LEFT_EAR.value],
                                        landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value],
                                        landmarks[mp_pose.PoseLandmark.LEFT_HIP.value])
        
        # 어깨 - 엉덩이 - 무릎 => 올리는 동작 각도 구하기 위함 -> Main angle
        angle = calculateAngle(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value],
                                            landmarks[mp_pose.PoseLandmark.LEFT_HIP.value],
                                            landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value])
        
        # 엉덩이 - 무릎 - 발목 => 무릎 꺾였는지 보기 위함
        left_knee_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.LEFT_HIP.value],
                                        landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value],
                                        landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value])

        if len(a) == 0 or len(a) == len(c) :
            if angle >140 : 
                print('다리 쭉 펴진 상태 -> 140~180도')
                # 이때의 고개 , 무릎 각도 저장
                a.append(left_knee_angle)
                a.append(left_shoulder_angle)
                stage='start'
        
        
        # 고점에 도착했을때            
        elif len(b) < len(a) :
            if angle < 110 :
                print('다리 들어 올린상태 -> 110도보다 작아질 정도로 올리기')
                b.append(left_knee_angle)
                b.append(left_shoulder_angle)
                stage = 'up'
        
        # 1회 완성 했을 때
        elif len(c) < len(b) :
            if angle >140 :
                print('다시 다시 내린 상태 -> 전과같이 140~180도')
                c.append(left_knee_angle)
                c.append(left_shoulder_angle)
                stage = 'complete'

                counter += 1 # 1회 동작 완료

                # 동작 완료 후 a,b,c 에 있는 각도 -> 한개의 행으로 합치기
                # -> 학습시켯던 모델의 X데이터가 됨
                globals()[f'f{counter}'] = a+b+c
                print(globals()[f'f{counter}'])
                # dataframe에 저장 -> 예측을 하기 위함
                df= df.append([globals()[f'f{counter}']])
                # 다시 abc 리스트 0으로 초기화
                a = []
                b = []
                c = []

In [ ]:
# 모델 불러서 result에 예측 하기
loaded_model = pickle.load(open('knnpicle_file', 'rb'))
result = loaded_model.predict(df) 

In [ ]:
# 각 회마다 맞는 자세인지 매핑하기
val=[]
for i in range(len(result)):
    if result[i] ==0:
        val.append('정자세')
    elif result[i] ==1:
        val.append('고개')
    elif result[i] == 2:
        val.append('다리')

In [ ]:
# val -> 한회마다의 자세 정보 ,len(val) -> 영상에서의 레그레이즈 횟수
val